In [5]:
# coding: utf-8

import time
import shutil
import sys
import io
import os
import csv
import re
import datetime
import mysql.connector
from  mysql.connector import MySQLConnection, Error

# 這麼多defualt是因為如果不重新設定，在使用 reload函數時，print會被印在 terminal上。
default_stdout = sys.stdout
default_stderr = sys.stderr
reload(sys)
sys.stdout = default_stdout
sys.stderr = default_stderr
sys.setdefaultencoding('utf-8')

cs = time.strftime('%Y/%m/%d_%H:%M:%S',time.localtime())
print cs
# 存粹看運算時間

def ETL():
    """
    把最原始得資料拿來做ETL，頭尾不用的去掉，加入新的欄位名稱。
    
    """
    try:
        # create 'clean' folder  if not exist.
        if not os.path.exists('/Users/Jackie/Desktop/data_create_insert_code_1107/clean'):
            os.makedirs('/Users/Jackie/Desktop/data_create_insert_code_1107/clean')
        # from infiark1/ pick only files to do the trick.
        files=[]
        for f in os.listdir('/Users/Jackie/Desktop/data_create_insert_code_1107/'):
            if os.path.isfile('/Users/Jackie/Desktop/data_create_insert_code_1107/' + f):
                files.append(f)
        # chose only the csv files to ETL and rename it.
        # tt=[]
        for f in files:
            if '.csv' in f:

                with io.open('/Users/Jackie/Desktop/data_create_insert_code_1107/' + f, 'r', encoding='utf-8')as rfile:
                    ff=[]
                    for ele in rfile:
                        ff.append(ele)
                    # 檔案名字用VD的編號，從檔案內的第一欄抓出來
                    fname = ff[2:-2][0].split(',')[0] 
                    with io.open('/Users/Jackie/Desktop/data_create_insert_code_1107/clean/' + \
                                 fname + '.csv', 'w', encoding='utf-8') as wfile:
                        wfile.write(u'''EQIPnumber,location,direct,YMD,hour,laneNumber,\
                        addTotal,15Total,truckflow,carflow,scooterflow,avgspeed,avgPercent,avgCarSpace''' + u'\n')
                        # 去掉檔案裡上下不用的中文敘述，再把平均速度='-1'的值挑掉不要，最後是把原來檔案內的','跟'\n'去掉。
                        for ele in ff[2:-2]:
                            if ele.split(',')[-4] == '-1.0':
                                continue
                            else:
                                sf=[]
                                for el in ele.split(','):
                                    if ele.split(',')[3] == el:
                                        sf.append(','.join((ele.split(',')[3]).split(' ')))
                                    else:
                                        sf.append(el)
                                wfile.write(','.join(sf)[:-2] + '\n')
                 
    except:
        with open('/Users/Jackie/Desktop/data_create_insert_code_1107/ETL.txt','w') as here3:
            here3.write(u'ETL exception'+u'\n')
            here3.write(str(sys.stderr)+u'\n')
            here3.write(str(sys.exc_info()[0])+u'\n')
            here3.write(str(sys.exc_info()[1])+u'\n')
            here3.write(str(sys.exc_info()[2])+u'\n')
        
                            
                            
def direct_add():
    """
    這個函數要做的只是把剛剛整理好的資料，
    因為有些VD的紀錄裡有一種或兩種方向或是根本沒指定方向，
    全部歸納成只有一種方向，並在檔案名稱上加入方向，
    東西南北就是EWSN，未指定就是U。
    
    """
    try:
        # create 'done' folder if not exist.
        if not os.path.exists('/Users/Jackie/Desktop/data_create_insert_code_1107/done'):
            os.makedirs('/Users/Jackie/Desktop/data_create_insert_code_1107/done')
        # from 'clean' folder chose file that is csv.
        files=[]
        for f in os.listdir('/Users/Jackie/Desktop/data_create_insert_code_1107/clean/'):
            if os.path.isfile('/Users/Jackie/Desktop/data_create_insert_code_1107/clean/'+f):
                files.append(f)
        for f in files:
            if '.csv' in f:
                # empty list to temp store data and seperate by direct
                Ntemp=[]
                Stemp=[]
                Etemp=[]
                Wtemp=[]
                Utemp=[]
                ff=[]
                with io.open('/Users/Jackie/Desktop/data_create_insert_code_1107/clean/'+f,'r',encoding='utf-8')as rfile:
                    for line in rfile:
                        ff.append(line)
                    for ele in ff[1:]:
                        #  有些資料可能會寫成'往北''往南'或'向北''向南'，但是都一樣方向
                        if (ele.split(',')[2] =='往北')or(ele.split(',')[2] =='向北')or(ele.split(',')[2] =='北'):
                            Ntemp.append(ele)
                        elif (ele.split(',')[2] =='往南')or(ele.split(',')[2] =='向南')or(ele.split(',')[2] =='南'):
                            Stemp.append(ele)
                        elif (ele.split(',')[2] =='往東')or(ele.split(',')[2] =='向東')or(ele.split(',')[2] =='東'):
                            Etemp.append(ele)
                        elif (ele.split(',')[2] =='往西')or(ele.split(',')[2] =='向西')or(ele.split(',')[2] =='西'):
                            Wtemp.append(ele)
                        elif (ele.split(',')[2] =='未指定')or(ele.split(',')[2] =='正向放置')\
                                                           or (ele.split(',')[2] =='反向放置'):
                            Utemp.append(ele) 
                # 這裡的寫法是因為空list的bool值是false，有值的[]是true，以此去判斷。
                if Ntemp:
                    with io.open('/Users/Jackie/Desktop/data_create_insert_code_1107/done/'+'N'+f,'w',encoding='utf-8') as nfile:
                        nfile.write(ff[0])
                        for ele in Ntemp:
                            nfile.write(ele)
                if Stemp:
                    with io.open('/Users/Jackie/Desktop/data_create_insert_code_1107/done/'+'S'+f,'w',encoding='utf-8') as sfile:
                        sfile.write(ff[0])
                        for ele in Stemp:
                            sfile.write(ele)
                if  Etemp:
                    with io.open('/Users/Jackie/Desktop/data_create_insert_code_1107/done/'+'E'+f,'w',encoding='utf-8') as efile:
                        efile.write(ff[0])
                        for ele in Etemp:
                            efile.write(ele)
                if Wtemp:
                    with io.open('/Users/Jackie/Desktop/data_create_insert_code_1107/done/'+'W'+f,'w',encoding='utf-8') as wfile:
                        wfile.write(ff[0])
                        for ele in Wtemp:
                            wfile.write(ele)
                if Utemp:
                    with io.open('/Users/Jackie/Desktop/data_create_insert_code_1107/done/'+'U'+f,'w',encoding='utf-8') as ufile:
                        ufile.write(ff[0])
                        for ele in Utemp:
                            ufile.write(ele)
    except:
        with open('/Users/Jackie/Desktop/data_create_insert_code_1107/direct_add.txt','w') as here3:
            here3.write(u'direct_add exception'+u'\n')
            here3.write(str(sys.stderr)+u'\n')
            here3.write(str(sys.exc_info()[0])+u'\n')
            here3.write(str(sys.exc_info()[1])+u'\n')
            here3.write(str(sys.exc_info()[2])+u'\n')
        
def insert_data():
    """
    主要是使用連結資料庫的第三方套件mysql.connector，
    以python去對資料庫做連結，在利用for loop把資料insert到正確的table下。
    
    """
    try:
#     連結資料庫的第三方套件
        cnx = mysql.connector.connect(user='root', password='apple', database='1108test')
        cursor = cnx.cursor()
        files = []
        for f in os.listdir('/Users/Jackie/Desktop/data_create_insert_code_1107/done/'):
            if os.path.isfile('/Users/Jackie/Desktop/data_create_insert_code_1107/done/' + f):
                files.append(f)
        replace2 = []
        for f in files:
            if '.csv' in f:
                #利用正規表達式，從檔名去找table名
                match = re.findall('[A-Z0-9]+', f)
                tablename = str(match[0])
                # 讀取要寫入的檔案
                with io.open('/Users/Jackie/Desktop/data_create_insert_code_1107/done/' + f, 'r', encoding='utf-8') as rcsv:
                    content = []
                    for line in rcsv:
                        content.append(line)
                    # 不要欄位名稱，所以要先將內容放在一個list，在取資料時，直接放棄第一筆[1:]
                    for element in content[1:]:
                        #不需要第一行標題 [1:]
                        #先用list再轉成tuple
                        mylist=[tablename,]
                        for ele in element.split(','):
                            mylist.append(str(ele).strip())
                        #轉成tuple
                        mytuple = tuple(mylist)
                        replace2.append(mytuple) 
        #這裡要注意的是'{r[1]}','{r[2]}'...等等到r5要加引號是因為在query時就要加''號，因為'00:00:00'沒加''會出錯！！！
        for ele in replace2:         
            query = ('''insert into {r[0]} (EQIPnumber,location,direct,YMD,hour,laneNumber,addTotal,15Total,
        truckflow,carflow,scooterflow,avgspeed,avgPercent,avgCarSpace)values
        ('{r[1]}','{r[2]}','{r[3]}','{r[4]}','{r[5]}',{r[6]},{r[7]},{r[8]},
        {r[9]},{r[10]},{r[11]},{r[12]},{r[13]},{r[14]});'''.format(r=ele))
            cursor.execute(query)
            cnx.commit()
        cursor.close()
        cnx.close()
    except:
        with open('/Users/Jackie/Desktop/data_create_insert_code_1107/insert_data.txt','w') as here3:
            here3.write(u'insert_data exception'+u'\n')
            here3.write(str(sys.stderr)+u'\n')
            here3.write(str(sys.exc_info()[0])+u'\n')
            here3.write(str(sys.exc_info()[1])+u'\n')
            here3.write(str(sys.exc_info()[2])+u'\n')

        
    

2016/11/09_18:42:28


In [6]:
def create_table():
    try:
        conn = mysql.connector.connect(host='localhost',
                                       database='1108test',
                                       user='root',
                                       password='apple')
        cursor = conn.cursor()
        files = []
        for f in os.listdir('/Users/Jackie/Desktop/data_create_insert_code_1107/done/'):
            if os.path.isfile('/Users/Jackie/Desktop/data_create_insert_code_1107/done/' + f):
                files.append(f)
        replace2 = []
        for f in files:
            if '.csv' in f:
                #利用正規表達式，從檔名去找table名
                match = re.findall('[A-Z0-9]+', f)
                tablename = str(match[0])
                replace2.append(tablename)
        for ele in replace2:         
            query = ('''create table {r} (EQIPnumber varchar(255),location varchar(255),direct varchar(25),
            YMD varchar(25),hour varchar(25),laneNumber int(2),addTotal double,15Total double,truckflow double,
            carflow double,scooterflow double,avgspeed double,avgPercent double,avgCarSpace double);'''.format(r=ele))
            cursor.execute(query)
            conn.commit()
    except Error as e:
        print(e)
        with open('/Users/Jackie/Desktop/data_create_insert_code_1107/create.txt','w') as here3:
            here3.write(u'create exception'+u'\n')
            here3.write(str(sys.stderr)+u'\n')
            here3.write(str(sys.exc_info()[0])+u'\n')
            here3.write(str(sys.exc_info()[1])+u'\n')
            here3.write(str(sys.exc_info()[2])+u'\n')
    finally:
        cursor.close()
        conn.close()
    

In [118]:
create_table()

In [7]:
ETL()
direct_add()




In [119]:
insert_data()

In [120]:
cs = time.strftime('%Y/%m/%d_%H:%M:%S',time.localtime())
print cs

2016/11/08_13:53:21
